# TP Relation Extraction

## Introduction

Dans ce TP nous allons:

1. Découvrir la tache d’extraction de relation par réseaux neuronaux  (« neural relation extraction », NRE)
2.  Comprendre le fonctionnement d’un modèle d’extraction de relation avec un encodeur BERT
3. Découvrir la tache de reconnaissance d'entités nommées (« named-entity recognition », NER)
4. Coder une pipeline d’extraction de relation par réseaux neuronaux pour des jeux de données textuels

Avec les outils suivants:
1. La libraire [OpenNRE](https://github.com/thunlp/OpenNRE), basée sur Pytorch et HuggingFace’s Transformers, pour la tache d’extraction de relation par réseaux neuronaux
2. [HuggingFace’s Transformers](https://huggingface.co/transformers/) : une bibliothèque basée sur Pytorch pour le traitement automatique des langues et notamment les modèles neuronaux de type Transformer (comme BERT)
3. Google Colab, qui héberge ce *Jupyter Notebook*. Avant de commencer le TP, vous pouvez consulter des pages d'introductions [à Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb#scrollTo=YHI3vyhv5p85) et [aux Notebooks](https://realpython.com/jupyter-notebook-introduction/)


Contrairement au dernier TP, cette fois-ci nous n’aurons pas besoin d’utiliser une GPU car nous n’allons pas entraîner des nouveaux modèles ou faire de l’inférence sur des grands jeux de données: une CPU suffira. Nous pouvons vérifier si l’on est en train d’utiliser une CPU ou une GPU avec les lignes suivantes:

In [ ]:
import torch

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

GPU is available.


Nous avons besoin d’installer l’outil OpenNRE. Pour éviter de devoir ré-télécharger le répertoire  GitHub de OpenNRE à chaque fois qu’on ré-initialise le fichier Colab, il est convenant de monter notre répertoire Google Drive et y télécharger le répertoire OpenNRE de façon à l’avoir toujours disponible. Ainsi, à chaque fois que nous allons ré-initialiser le fichier Colab, il nous suffira de monter notre Goodle Drive pour avoir accès à la libraire OpenNRE.

> ATTENTION: modifiez *tp_path_in_drive* pour pointer vers le repertoire où vous avez placé le fichier tp_re.ipynb, vous allez télécharger OpenNRE dans le meme réépertoire. Si vous êtes sur votre machine locale, vous n'avez pas besoin de monter le Drive, mais juste de faire le clonage du réépertoire OpenNRE.

In [ ]:
import os
from google.colab import drive

tp_path_in_drive = '/content/drive/My Drive/TAL/tp_re'
opennre_path_in_drive = tp_path_in_drive + '/OpenNRE'

# mount Google Drive
drive.mount('/content/drive')

if not os.path.isdir(opennre_path_in_drive):
  # OpenNRE is not already present in Google Drive
  if not os.path.isdir(tp_path_in_drive):
    # make directory for the TP if necessary
    os.makedirs(tp_path_in_drive, exist_ok=True)
  # change directory to the TP directory
  os.chdir(tp_path_in_drive)
  # clone OpenNRE repo
  print("Cloning repo...")
  os.system('git clone https://github.com/thunlp/OpenNRE.git')
  print("...done!")
else:
  print("OpenNRE is already present in Google Drive under {0}".format(opennre_path_in_drive))

# Change current dir to OpenNRE
os.chdir(opennre_path_in_drive)

Mounted at /content/drive
OpenNRE is already present in Google Drive under /content/drive/My Drive/TAL/tp_re/OpenNRE


In [ ]:
# Update requirements
!sed -i '/transformers==3.0.2/c\transformers==3.4.0' requirements.txt

Nous pouvons désormais continuer avec l’installation.

In [ ]:
!pip install -r requirements.txt
!python setup.py install

     |████████████████████████████████| 748.8 MB 20 kB/s 
     |████████████████████████████████| 1.3 MB 39.4 MB/s 
     |████████████████████████████████| 234 kB 50.4 MB/s 
     |████████████████████████████████| 7.0 MB 37.8 MB/s 
     |████████████████████████████████| 1.5 MB 49.4 MB/s 
     |████████████████████████████████| 1.2 MB 36.4 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 2.9 MB 30.1 MB/s 
     |████████████████████████████████| 749 kB 36.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+

running install
running bdist_egg
running egg_info
writing opennre.egg-info/PKG-INFO
writing dependency_links to opennre.egg-info/dependency_links.txt
writing top-level names to opennre.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'opennre.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying build/lib/opennre/encoder/__init__.py -> build/bdist.linux-x86_64/egg/opennre/encoder
copying build/lib/opennre/encoder/base_encoder.py -> build/bdist.linux-x86_64/egg/opennre/encoder
creating build/bdist.linux-x86_64/egg/opennre/framework
copying build/lib/opennre/framework/utils.py -> build/bdist.linux-x86_64/egg/opennre/framework
copying build/lib/opennre/framework/data_loader.py -> build/bdist.linux-x86_64/egg/opennre/framework
copying build/lib/opennre/framework/multi_label_sentence_re.py -> build/bdist.linux-x86_64/egg/opennre/framework
copying build/lib/opennre/framework/__init__.py -> build/b

## Les modéles pour l'extraction de relation

À part offrir un cadre pour l’implémentation et l’entraînement de modelés pour l’extraction de relation, OpenNRE offre aussi des modèles déjà entraînés sur différents jeux de données, et donc capables de détecter différents types de relations entre entités. 

Ici, nous allons employer un modèle entraîné sur Wiki80 (dataset introduit par [le papier OpenNRE](https://www.aclweb.org/anthology/D19-3029/)), un jeu de données contenant des phrases collectées sur Wikipedia et Wikidata, ainsi que des rélations entre leurs entités. Si vous voulez en savoir plus sur Wiki80, vous pouvez le télécharger avec le script [download_wiki80.sh](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/benchmark/download_wiki80.sh) 

In [ ]:
import opennre
model = opennre.get_model('wiki80_bert_softmax')

2022-03-14 09:02:15,771 - root - INFO - Loading BERT pre-trained checkpoint.


Nous pouvons utiliser ce modèle pour calculer la relation entre un mot «tête» et un mot «queue» qui sont contenus dans un texte. Il suffit de passer au modèle le texte ainsi que la position de la tête et de la queue. Le modèle retournera la relation de la queue à l’égard de la tête, ainsi que la probabilité qu’il associe à cette rélation. Par example, nous pouvons inféérer la relation entre **Áed Uaridnach** et **Máel Dúin mac Máele Fithrich** de la façon suivante:

In [ ]:
item = {'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', 'h': {'pos': (78, 91)}, 't': {'pos': (18, 46)}}
print(model.infer(item))

('child', 0.9812852144241333)


#### Exercice 1



Écrire une fonction `to_input_format(text, head, tail)` qui nous permet de trouver la position de deux mots (une tête et une queue) dans un texte, et qui retourne un dictionnaire contenant le texte et les deux positions suivant le format requis par la fonction `model.infer()`.

In [ ]:
import re

def to_input_format(text, head, tail):
  """
  Args:
    text: a string of text
    head: a string of text representing a word contained in text
    tail: a string of text representing a word contained in text but different from head
  Returns:
    A dictionary containing the text and the position of head and tail within it,
    following the input format required by a SoftmaxNN model.
  """
  start_index_head = text.find(head)
  end_index_head = start_index_head + len(head)
  start_index_tail = text.find(tail)
  end_index_tail = start_index_tail + len(tail)
  item = dict()
  item['text'] = text
  item['h'] = {'pos': (start_index_head, end_index_head)}
  item['t'] = {'pos': (start_index_tail, end_index_tail)}
  return item

In [ ]:
# Test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
head = 'Áed Uaridnach'
tail = 'Máel Dúin mac Máele Fithrich'
test_item = to_input_format(text, head, tail)
try:
  assert model.infer(item) == model.infer(test_item)
  print("Good job!")
except AssertionError:
  print("Something is wrong with your function, try again!")

Good job!


#### Exercice 2

Décrire l’architecture du modèle qu’on vient de télécharger, la logique et le fonctionnement de chacun de ses composants:

1. Sentence-encoder: pourquoi l'on utilise un encodeur type BERT ?
2. À quoi sert la couche BertPooler ?
3. À quoi sert la couche linéaire finale (fc) ? pourquoi elle réduit à 80 la dimension du vecteur sortant de l’encodeur?
4. À quoi sert la fonction Softmax ?
5. À quoi sert la fonction de dropout que l'on appliuque à la sortie du réseau, ainsi que dans chaque couche de BERT ?

Aide: vous pouvez inspecter un modèle Pytorch avec `print(model)`. Pour mieux le comprendre, vous pouvez aussi voir [son code](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/model/softmax_nn.py#L5), [la déscription du modèle BERT de Huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/model_doc/bert#transformers.BertModel) ainsi que [son code](https://github.com/huggingface/transformers/blob/v4.16.2/src/transformers/models/bert/modeling_bert.py#L848).

In [ ]:
print(model)

SoftmaxNN(
  (sentence_encoder): BERTEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
        

1. On utilise un encoder bert car afin de prédire une relation entre deux mots, il faut prendre le contexte complet de la phrase et ainsi avoir une approche bidirectionnelle

2. Le BertPooler renvoie le premier token passé par un modèle linéaire et avec une fonction d'activation non linéaire de Tanh.

3. Pass

4. Le softmax a permet d'obtenir une probabilité (c'est une transformation qui renvoie une suite de réels > 0 et qui somment à 1) **dans notre cas, quelle est la significaiton de la distribution de probabilité renvoyée par la softmax? -> Le softmax renvoie la probabilité associée à chacune des 80 classes (rélations entre head-tail) possibles**

5. Les différents drop permettent d'activer ou d'eteindre une partie du réseau avec une probabilité donnée. Cela permet d'éviter l'overfitting des données. **ce n'est pas très claire comment le dropout aide le modèle à ne pas overfitter -> le fait que les échantillons soient pérturbés empêche au modèle de "les apprendre par coeur". Ainsi, le modèle est encouragé à apprendre à généralizer par rapport aux exemples d'entrainement.**

## Encodeur

#### Exercice 3

Maintenant, nous allons essayer de mieux comprendre le fonctionnement de ce modèle avec un focus sur son encodeur, qui est définit dans le fichier [bert_encoder.py](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/encoder/bert_encoder.py#L7). Ce qui est spécifique à la tache de NRE dans ce modèle n’est pas l’architecture, mais plutôt la façon dont la séquence en input est tokenisée. Avec l’objectif de bien comprendre comment cet encodeur gère son input, répondez en détail aux questions suivantes qui se référent à la méthode `tokenize`. Si vous utilisez du code pour vous aider à répondre (conseillé), veuillez le joindre à vos réponses textuelles.

1. Qu’est-ce qu’elle sont les variables `sent0`, `ent0`, `sent1`, `ent1`, `sent2` ?
2. Qu’est-ce que c’est `re_tokens` ?
3. Dans le _forward_, le `BERTEncoder` prend en entrée exclusivement la séquence textuelle qui a étée préalablement tokenisée. Comment est-il capable de distinguer la tête et la queue en sort de (apprendre à) prédire la relation entre les deux ?
4. Qu’est-ce que c’est le « Padding » et pourquoi est-il utile ?
5. Qu’est-ce que c’est l’ « Attention mask » et pourquoi est-elle utile ? [Aide ici](https://huggingface.co/docs/transformers/v4.16.2/en/glossary#attention-mask)
6. **[BONUS]** Quelle est la différence entre la classe `BERTEncoder` et la classe `BERTEntityEncoder`, définie dans le même fichier ?


Aide : vous pouvez accéder à la méthode `tokenize` pour la tester avec `model.sentence_encoder.tokenize(item)`

In [ ]:
model.sentence_encoder.tokenize(item)

(tensor([[  101,  2002,  2001,  1996,  2365,  1997,     3, 11530,  2140,  4241,
           2378,  6097, 11530,  2571,  4906, 26378,  2818,     4,  1010,  1998,
           7631,  1997,  1996,  2152,  2332,     1, 29347,  2094, 25423, 14615,
          18357,  2232,     2,  1006,  2351,  6079,  2475,  1007,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0, 0, 0, 0]]),
 tensor([[25]]),
 tensor([[6]]))

1. Ces variables permettent de décomposer la phrase en plusieurs éléments. Ainsi, sent0 contient la première partie de la phrase jusqu'à la première entité (head ou tail), ent0 contient la première entité, sent1 la partie de la phrase entre la fin de la première entité et le début de la seconde, ent1 la seconde entité et sent2 la partie de la phrase entre la fin de la seconde entité et la fin de la phrase.

2. re_tokens permet d'avoir la phrase tokenisé en ajoutant tous ces éléments et les tokens CLS et SEP

3. Les variables pos1 et pos2 sont peut-être utilisées pour retrouver les sentinelles et entités dans la phrase tokenisée.

4. Le padding représente des tokens vides car le modèle de BERT reçoit des tailles de phrases fixes, il faut donc ajouter du padding si la phrase est trop courte. **Pourquoi est-il utile d'uniformer la taille? -> pour qu'on puisse representer un lot de phrases avec une matrice, ce qui est nécessaire au réseau pour traiter les phrases en parallèle.**

5. L'attention mask permet de savoir s'il faut tenir compte de certains tokens (1 ou 0), cela est donc utile avec le padding par exemple pour ne pas le prendre en compte.

## NRE Pipeline

Nous allons maintenant programmer une application qui prend en entrée une phrase et donne en sortie deux entités nommées dans la phrase ainsi que la relation entre eux.

Pour ce faire, nous avons besoin :

1. d’un système NER, qui reconnaît les entités nommées dans la phrase
2. d’un système NRE, comme celui que nous avons utilisé jusqu’à là

#### Exercice 4

En vous aident avec la documentation de HuggingFace, instanciez une [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipeline#the-pipeline-abstraction) `ner_pipeline` pour la reconnaissance d'entités nommées, avec le modèle et le tokeniseur pré-entraînes [dbmdz/bert-large-cased-finetuned-conll03-english](https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).

Vous pouvez tester la bonne réussite de l’exercice avec le code ci-dessous.

**Indice** : la solution consiste en deux lignes de code : l'une pour importer la classe pipeline, l'autre pour instancier la bonne pipeline.

In [ ]:
from transformers import pipeline
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
# Apply your ner_pipeline to some sentences to see how it works,
# Then you can test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
try:
  assert ner_pipeline(text) == [
                                {'entity': 'I-PER', 'index': 6, 'score': 0.982085645198822, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 7, 'score': 0.8588601350784302, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 8, 'score': 0.8766007423400879, 'word': '##el'},
                                {'entity': 'I-PER', 'index': 9, 'score': 0.8375428915023804, 'word': 'D'},
                                {'entity': 'I-PER', 'index': 10, 'score': 0.4897182583808899, 'word': '##ú'},
                                {'entity': 'I-PER', 'index': 11, 'score': 0.901253342628479, 'word': '##in'},
                                {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
                                {'entity': 'I-PER', 'index': 13, 'score': 0.8884768486022949, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 14, 'score': 0.7004077434539795, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 15, 'score': 0.8791088461875916, 'word': '##ele'},
                                {'entity': 'I-PER', 'index': 16, 'score': 0.9720047116279602, 'word': 'Fi'},
                                {'entity': 'I-PER', 'index': 17, 'score': 0.697806715965271, 'word': '##th'},
                                {'entity': 'I-PER', 'index': 18, 'score': 0.697088360786438, 'word': '##rich'},
                                {'entity': 'I-PER', 'index': 26, 'score': 0.9203088283538818, 'word': 'Á'},
                                {'entity': 'I-PER', 'index': 27, 'score': 0.9416706562042236, 'word': '##ed'},
                                {'entity': 'I-PER', 'index': 28, 'score': 0.9702795147895813, 'word': 'U'},
                                {'entity': 'I-PER', 'index': 29, 'score': 0.8428962230682373, 'word': '##ari'},
                                {'entity': 'I-PER', 'index': 31, 'score': 0.7077912092208862, 'word': '##ach'}
                              ]
  print("Good job!")
except AssertionError:
  print("Something might be wrong with your pipeline.") 

Good job!


#### Exercice 5


Nous pouvons finalement développer une pipeline NRE reposante sur notre modèle de NRE et notre pipeline NER. 

Écrivez ci-dessous une classe `NREPipeline` équipée (entre autres) d'une méthode `__call__(self, text)` qui prend un texte en entrée et effectue les opérations suivantes :

- elle reconnaît les entités dans le texte
    - retenir seulement les deux entités auxquelles la pipeline NER associe la probabilité la plus élevée, écarter les autres (si presentes)
    - si la pipeline NER ne reconnaît qu’une seule entités dans le texte, `__call__(self, text)` retourne None (voir le test en bas) car il n’y a aucune relation à prédire
- elle donne en sortie une liste en format `[e1, e2, rel, p]` où :
    - `e1` est la première entité reconnue dans le texte (entre les deux plus probables, la première qui apparaître dans le texte)
    - `e2` est la deuxième entité  reconnue dans le texte (entre les deux plus probables, la deuxième qui apparaître dans le texte)
    - `rel` est la relation qu’il y a entre `e1` et `e2`
    - `p` est la probabilité associée à la relation `rel` par le modèle de NRE

Pour vérifier la bonne qualité de votre classe `NREPipeline`, utilisez l’extrait de code ci-dessous. Le résultat devrait ressabler à celui-ci :

````
Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['I', 'World Heritage Site', 'heritage designation', 0.9991846680641174]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'location', 0.9127373099327087]
------------------------------
````

In [ ]:
# Write your pipeline in this cell. The fun begins here, because we are coding a whole Python class from scratch !

class NREPipeline(object):
  def __init__(self, ner_pipeline, nre_model):
    self.ner_pipeline = ner_pipeline
    self.ner_pipeline.grouped_entities=True
    self.nre_model = nre_model

  # ...


  def __call__(self, text):
    predict = ner_pipeline(text)
    predict = [item for item in predict if item['word'] != '[UNK]'] #supprime si le mot est [UNK]
    if len(predict) > 1:
      first = max(predict, key=lambda p: p['score'])
      predict.remove(first)
      second = max(predict, key=lambda p: p['score'])
      if text.find(first['word']) < text.find(second['word']) :
        e1 = first['word']
        e2 = second['word']
      else :
        e1 = second['word']
        e2 = first['word']
      relation = model.infer(to_input_format(text, e1, e2))
      return [e1, e2, relation[0], relation[1]]
    else:
      return None


In [ ]:
# Test your code with this snippet

sequences = [
             'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', # Easy sentence
             'He was the son of Máel Dúin', # There is only one entity in this sentence, therefore our pipeline should return None
             'Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .', # Ōda is tokenized by the NER tokenizer as a "[UNK]" and it is detected as an entitity. For simplicity, you can discard [UNK] entities as if they were not detected. Here another difficulty arises from the fact that the different tokens in which Iwami Ginzan is decomposed are (wrongly) classified as entities belonging to different classes. In this case, we can consider only the first token "I" as standalone entity, for simplicity.  
             'It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .', # the NER system can not recognise any entity here, therefore the pipeline should return None
             "Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church ." # This sentence is not trivial because the 's has to be managed properly

]

nre = NREPipeline(ner_pipeline, model)
for n, sequence in enumerate(sequences):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['##zan', 'World Heritage Site', 'heritage designation', 0.9972099661827087]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ["St . Stephen ' s Episcopal Church", 'Earleville', 'located in the administrative territorial enti

## Application de le pipeline

#### Exercice 6

1. Appliquez la pipeline NRE aux phrases contenues dans le fichier _sentences.txt_, qui ont été extraites à partir de Wikipedia et Wikidata. Donnez un avis qualitatif sur sa performance, les problèmes rencontrés ainsi que des idées pour améliorations des résultats (e.g. entraînement sur des données différentes, modèle différent, etc.).

2. Appliquez la pipeline NRE aux premières 50 phrases contenues dans le fichier _Sentences_AllAgree.txt_ que vous avez utilisé pour le TP BERT, qui parlent d’événements financiers. Remarquez-vous des différences en terme de performances par rapport à la question 2 ? Pourquoi ? Commentez…

In [ ]:
# Question 1
file = open('../sentences.txt', 'r')
for n,sequence in enumerate(file.readlines()):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: The Willard and Josephine Hubbard House was individually listed on the National Register of Historic Places in 2016 .

System out:  ['Willard', 'Josephine Hubbard', 'spouse', 0.9874956607818604]
------------------------------
Sentence 1: His station commander , Group Captain Claude Hilton Keith , found a letter among the missing airman 's personal possessions .

System out:  None
------------------------------
Sentence 2: One was Quintus Caecilius Metellus Creticus , who was praetor in 74 BC and consul in 69 BC .

System out:  None
------------------------------
Sentence 3: April 2009 In addition to musical acts , the label recorded beat poets Lawrence Ferlinghetti and Allen Ginsberg and comic Lenny Bruce .

System out:  ['Allen Ginsberg', 'Lenny Bruce', 'sibling', 0.7926136255264282]
------------------------------
Sentence 4: Walter Neusel ( November 25 , 1907 – October 3 , 1964 ) was a German heavyweight boxer .

System out:  ['Walter Neusel', 'German', 'country of citize

Les résultats sont plutôt bons, il y a quelques associations fausses mais les probabilités associées représentent bien l'exactitude des relations. Les deux entités correspondent assez bien.

In [ ]:
# Question 2
file = open('../Sentences_AllAgree.txt', encoding="utf-8", errors = 'ignore')
for n,sequence in enumerate(file.readlines()):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')
  if n == 49:
    break

Sentence 0: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral

System out:  ['Gran', 'Russia', 'owned by', 0.881830096244812]
------------------------------
Sentence 1: For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .@positive

System out:  ['Componenta', 'EU', 'applies to jurisdiction', 0.4356616139411926]
------------------------------
Sentence 2: In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .@positive

System out:  ['EU', 'EU', 'member of', 0.6252642869949341]
------------------------------
Sentence 3: Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .@positive

System out:  ['EU', 'EU', 'applies to jurisdic

Les performances sont ici moins bonnes, il y a beaucoup de propositions avec des pourcentages plus faibles mais le principale problème vient aussi du choix des entités. Il y a par exemple beaucoup de 'EU' associé à 'EU' à cause des prix en euros. Les devises posent donc un problème car ils représentent moins des mots mais apparaissent plusieurs fois.